In [1]:
import os, glob
from arcgis.gis import GIS 
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import zipfile

In [2]:
load_dotenv()  # take environment variables from .env.settings

True

In [4]:
ARCGIS_CLIENT_ID = os.getenv("ARCGIS_CLIENT_ID")
ARCGIS_CLIENT_SECRET = os.getenv("ARCGIS_CLIENT_SECRET")
INPUT_DATA = 'F:\\TAM\\Data\\Road_Inventory_Geodatabase.zip'
gis = GIS("https://pritom.maps.arcgis.com/home", client_id=ARCGIS_CLIENT_ID, client_secret=ARCGIS_CLIENT_SECRET)
gis

GIS @ https://pritom.maps.arcgis.com version:2025.3

In [5]:
item = gis.content.get('ceca2d40bf29493f99b404c83216a9ee')  # Road Inventory Geodatabase
item

<Item title:"Road_Inventory_Geodatabase" type:Feature Layer Collection owner:Walter_Heisenberg>

In [6]:
item.layers[0]

<FeatureLayer url:"https://services7.arcgis.com/fXrkwuwpWhPZsN0b/arcgis/rest/services/Road_Inventory_Geodatabase/FeatureServer/0">

In [7]:
item.layers[0].properties

{
  "currentVersion": 11.5,
  "id": 0,
  "name": "COUNTY_ID",
  "type": "Feature Layer",
  "serviceItemId": "ceca2d40bf29493f99b404c83216a9ee",
  "cacheMaxAge": 30,
  "displayField": "",
  "description": "",
  "copyrightText": "",
  "defaultVisibility": true,
  "editingInfo": {
    "lastEditDate": 1768846552684,
    "schemaLastEditDate": 1768846552684,
    "dataLastEditDate": 1768846552684
  },
  "relationships": [],
  "isDataVersioned": false,
  "hasContingentValuesDefinition": false,
  "supportsAppend": true,
  "supportsCalculate": true,
  "supportsASyncCalculate": true,
  "supportsTruncate": true,
  "supportsAttachmentsByUploadId": true,
  "supportsAttachmentsResizing": true,
  "supportsRollbackOnFailureParameter": true,
  "supportsStatistics": true,
  "supportsExceedsLimitStatistics": true,
  "supportsAdvancedQueries": true,
  "supportsValidateSql": true,
  "supportsCoordinatesQuantization": true,
  "supportsLayerOverrides": true,
  "supportsTilesAndBasicQueriesMode": true,
  "supp

In [9]:
features = item.layers[0].query(where="1=1", out_fields='*', return_geometry=True)
features.sdf.head()

,OBJECTID,RouteId,BeginPoint,EndPoint,COUNTY_IDVn,COUNTY_IDVt,COUNTY_IDVd,COUNTY_IDComments,Shape__Length,SHAPE
0,767,1000100002000INC,77.5048,77.5219,57.0,6,<NA>,<NA>,33.271754,"{""paths"": [[[-9379665.56703818, 4060672.371728..."
1,1898,1000100011000INC,30.4062,30.424,39.0,5,<NA>,<NA>,33.603393,"{""paths"": [[[-9104619.34388261, 3652494.411053..."
2,1961,1000100000600INC,71.9958,72.0134,121.0,7,<NA>,<NA>,34.034754,"{""paths"": [[[-9400924.02655412, 3981226.790954..."
3,2020,1000100008700INC,33.8158,33.8324,23.0,2,<NA>,<NA>,31.657927,"{""paths"": [[[-9276509.05491134, 3815869.802687..."
4,2235,1000100002700INC,61.632,61.6493,261.0,3,<NA>,<NA>,32.942634,"{""paths"": [[[-9373964.14918351, 3772570.040027..."


In [13]:
# 1. Query the layer (where="1=1" gets all features)
feature_set = item.layers[0].query(where="1=1", out_fields='*', return_geometry=True)
df = feature_set.sdf

# 2. Map the data to your specific dashboard format
asset_inventory = []

for index, row in df.iterrows():
    # Note: Using .get() ensures the code doesn't crash if a field is missing
    asset_inventory.append({
        # Ensure 'RouteId' vs 'Route_ID' matches your layer's schema
        "route_id": row.get('RouteId', 'N/A'), 
        "classification": row.get('F_SYSTEM', 'N/A'),
        "surface_type": row.get('SURFACE_TYPE', 'N/A'),
        "length": round(row.get('Shape__Length', 0) / 1000, 2), # Convert meters to KM if needed
        "shoulder_width": row.get('SHOULDER_WIDTH_R', 0),
        "lanes": row.get('THROUGH_LANES', ),
        "pavement_type": row.get('SURFACE_TYPE', 'ASPHALT'),
        "aadt": int(row.get('AADT', 0)) if row.get('AADT') else 0,
        "installation": row.get('Year_Built', '2010'),
        "speed_limit": row.get('Speed_Lim', 45),
        "gradient": 0,
        "cracking_percent": 0,
        "year_last_improvement": row.get('Last_Improvement', 'N/A'),
        "thickness": 0
    })

# 3. View the first 5 records of your mapped data
import json
print(json.dumps(asset_inventory[:5], indent=2))

[
  {
    "route_id": "1000100023300INC",
    "classification": "N/A",
    "surface_type": "N/A",
    "length": 0.0,
    "shoulder_width": 0,
    "lanes": null,
    "pavement_type": "ASPHALT",
    "aadt": 0,
    "installation": "2010",
    "speed_limit": 45,
    "gradient": 0,
    "cracking_percent": 0,
    "year_last_improvement": "N/A",
    "thickness": 0
  },
  {
    "route_id": "1000100010400INC",
    "classification": "N/A",
    "surface_type": "N/A",
    "length": 0.0,
    "shoulder_width": 0,
    "lanes": null,
    "pavement_type": "ASPHALT",
    "aadt": 0,
    "installation": "2010",
    "speed_limit": 45,
    "gradient": 0,
    "cracking_percent": 0,
    "year_last_improvement": "N/A",
    "thickness": 0
  },
  {
    "route_id": "1000100003100INC",
    "classification": "N/A",
    "surface_type": "N/A",
    "length": 0.0,
    "shoulder_width": 0,
    "lanes": null,
    "pavement_type": "ASPHALT",
    "aadt": 0,
    "installation": "2010",
    "speed_limit": 45,
    "gradient":

In [9]:
# List all layers in your uploaded item to find the one with road data
for i, layer in enumerate(item.layers):
    print(f"Index {i}: {layer.properties.name}")

# Once you find the correct index (e.g., index 1), update your query:
target_layer = item.layers[1] # Change index as needed
df = target_layer.query(where="1=1", out_fields='*').sdf

Index 0: COUNTY_ID
Index 1: F_SYSTEM
Index 2: LANE_WIDTH
Index 3: MEDIAN_WIDTH
Index 4: MEDIAN_TYPE
Index 5: NHS
Index 6: OWNERSHIP
Index 7: SHOULDER_TYPE
Index 8: SHOULDER_WIDTH_L
Index 9: SHOULDER_WIDTH_R
Index 10: STRAHNET_TYPE
Index 11: SURFACE_TYPE
Index 12: THROUGH_LANES
Index 13: URBAN_ID
Index 14: FACILITY_TYPE
Index 15: GA_2024_Routes


In [12]:
# 1. Target the correct layer
data_layer = item.layers[15] # GA_2024_Routes

# 2. Use a field that exists in THIS layer (e.g., RouteId)
# Instead of COUNTY_IDVn, use a simple wildcard or a known RouteId
where_clause = "1=1" 

# 3. Query with a limit to prevent timeouts
feature_set = data_layer.query(
    where=where_clause,
    out_fields='*',
    result_record_count=10, # Start small to verify fields
    return_geometry=True
)

# 4. Convert to DataFrame and check columns
df = feature_set.sdf
print("Actual fields in this layer:", df.columns.tolist())

Actual fields in this layer: ['OBJECTID', 'FUNCTION_TYPE', 'COUNTY', 'SYSTEM_CODE', 'DIRECTION', 'ROUTE_ID', 'Comments', 'StateID', 'BeginDate', 'START_M', 'END_M', 'FROM_MILEPOINT', 'TO_MILEPOINT', 'BeginPoint', 'EndPoint', 'RouteId', 'Shape__Length', 'SHAPE']
